### Muhammed Enes Baysan - 95180012 / YBS3
### Asrın İrem Meral - 95180057 / YBS3

In [373]:
import pandas as pd
import numpy as np

In [374]:
#dataset = {
#    'Gender':['F','F','F','F','M','F','F','M','M','F','F','M','M','M','M'],
#    'Weight':[1,1,1,2,2,2,2,2,3,3,3,3,4,4,5],
#    'Height':[2,1,1,2,1,1,2,3,2,3,2,2,3,4,4],
#    'Size':['M','S','M','M','S','M','S','M','M','L','M','S','M','L','L']
#}

In [375]:
#df = pd.DataFrame(dataset)

In [376]:
#df.to_csv('data.csv')

# Veri setini okuma

In [377]:
df = pd.read_csv('data.csv')
df.drop('Unnamed: 0',axis=1,inplace=True)

In [378]:
df.head()

,Gender,Weight,Height,Size
0,F,1,2,M
1,F,1,1,S
2,F,1,1,M
3,F,2,2,M
4,M,2,1,S


# Bayes Fonksiyonu

In [379]:
def bayes(df,target_feature,x,result_type):
    ################# Bayes Formulü #################
    # P(Cj \ xi) = (P(xi \ Cj) * P(Cj)) / P(xi)
    ##################################
    ################# Bağımlı ve Bağımsız değişkenleri ayırıyoruz #################
    independent_variables = [] # bağımsız değişkenlerimizin adını saklayacağımız liste
    for col in df.columns:
        if col != target_feature: # eğer gelen veri setindeki kolonun adı bağımlı değişken ile aynı değilse listeye at
            independent_variables.append(col)
    X = df.loc[:,independent_variables] # hedef feature'ı tahmin etmek için kullanacağımız değişkenler (bağımsız) (df olarak seçer)
    y = df.loc[:,[target_feature]] # target feature değişkenimiz (bağımlı) (df olarak seçer)
    ##################################
    ################# Hedef değişkenimizdeki her sınıfın olasılığını hesaplıyoruz (P(Cj)) #################
    feature_probabilities = {}
    for feature in y[target_feature].unique():
        feature_probabilities[feature] = y[y[target_feature] == feature].shape[0] / y.shape[0] # Her sınıfın olma olasılığını buluyoruz (P(Cj))
    ##################################
    ################# Gelen x değerinin her sınıfta olma olasılıklarını buluyoruz (P(xi \ Cj)) #################
    x_probabilities = {}
    for feature in y[target_feature].unique(): # hedef değişkenimizdeki (bağımlı) tüm sınıfları dönüyoruz
        counter = 0 # bağımsız değişkenleri tuttuğumuz listedeki elemanlara index ile erişmek istediğimiz için bir counter oluşturuyoruz
        query = f"({target_feature} == '{feature}') & " # df'i filtrelemek için bir sorgu oluşturuyoruz
        for iv in independent_variables: # her bir bağımsız değişkeni her bir bağımlı değişken sınıfı için dönüyoruz
            if independent_variables.index(iv) + 1 != len(independent_variables): # eğer döndüğümüz bağımsız değişken listenin son elemanı değilse sorgunun sonuna & (and) ekliyoruz
                query += f"({iv} == '{x[counter]}') & "
            else: # eğer son eleman ise sorgunun sonuna & (and) eklemiyoruz
                query += f"({iv} == '{x[counter]}')"
            counter+=1 # her döndüğümüzde bir sonraki bağımsız değişkene ulaşmak için counter'ı 1 artırıyoruz
        x_prob = df.query(f"{query}").shape[0] / y[y[target_feature] == feature].shape[0]
        x_probabilities[feature] = x_prob
    ##################################
    ################# Gelen x değerinin olma olasılığını buluyoruz (xi) #################
    xi_probabilities = {}
    xi_probability = 0
    for i in feature_probabilities:
        xi_probabilities[i] = feature_probabilities[i] * x_probabilities[i] # her bir hedef değişken sınıfının olma olasılığı ile o sınıf varken gelen x değerlerinin olma olasılıklarını buluyoruz
    for i in xi_probabilities:
        xi_probability+= xi_probabilities[i] # her bir olasılığı toplayıp xi'nin olma olasılığını buluyoruz
    ##################################
    if result_type == 'probability':
        return xi_probability
    elif result_type == 'probabilities':
        return xi_probabilities
    else:
        ex = f'Lütfen result_type parametresini geçerli bir değer olarak gönderin:\n*x* değerinin olma olasılığı için probability\n*x* değerinin hangi {target_feature} içerisinde olabileceğini görmek için probabilities'
        raise Exception(ex)

In [380]:
a1 = bayes(df,'Size',['M',3,2],'probabilities')

In [381]:
a1

{'M': 0.06666666666666667, 'S': 0.06666666666666667, 'L': 0.0}